In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
from pydantic import BaseModel
import numpy as np

from stock import kabutan

In [ ]:
json_dir = Path("../data/catalyst")

json_file_list = sorted(json_dir.glob("*.json"))

data_list = []
for json_file in json_file_list:
    with open(json_file, "r") as f:
        data_list.append(kabutan.CatalystStocks.model_validate_json(f.read()))

len(data_list) 

In [ ]:
class CatalystDailySummary(BaseModel):
    day: str
    good_up_rates: list[float]
    bad_up_rates: list[float]
    mid_up_rates: list[float]

summary_list = []
for data in data_list:
    summary = CatalystDailySummary(
        day = data.date,
        good_up_rates = [stock.up_rate for stock in data.stocks if stock.category=="good"],
        bad_up_rates = [stock.up_rate for stock in data.stocks if stock.category=="bad"],
        mid_up_rates = [stock.up_rate for stock in data.stocks if stock.category=="mid"],
    )
    summary_list.append(summary)

In [ ]:
def moving_average(array, cnt):
    filter = np.ones(cnt * 2 - 1)
    filter[cnt:] = 0.0
    return np.convolve(array, filter, "same") / cnt


def show_mean_summary(summary_list, target_key = "good_up_rates"):
    ups = []
    downs = []
    evens = []
    for summary in summary_list:
        for up_rates in summary.__dict__[target_key]:
            if up_rates > 0:
                ups.append(up_rates)
            elif up_rates < 0:
                downs.append(up_rates)
            else:
                evens.append(up_rates)

    total = len(ups) + len(downs) + len(evens)
    print("{} catalylsts : num up : {}, num down : {}, num even : {}".format(
        target_key.split("_")[0],
        len(ups), len(downs), len(evens)
    ))
    print("Up prob : {:.2f}, (mean : {:.3f}), Down prob : {:.2f}, (mean : {:.3f}), Even prob : {:.2f}".format(
        len(ups) / total, sum(ups) / len(ups), 
        len(downs) / total, sum(downs) / len(downs),
        len(evens) / total
    ))

    mean_list = [sum(d.__dict__[target_key]) / len(d.__dict__[target_key]) if len(d.__dict__[target_key]) > 0 else 0.0 for d in summary_list ]
    mv_average5 = moving_average(mean_list, 5)
    mv_average10 = moving_average(mean_list, 10)

    total = 1.0
    for val in mean_list:
        total = total * (1.0 + val)
    print(f"Total :: {total}")
    
    plt.figure(figsize=(20, 10))
    plt.plot(mean_list)
    plt.plot(mv_average5)
    plt.plot(mv_average10)
    plt.grid()
    return np.array(mean_list)

In [ ]:
good_mean_list = show_mean_summary(summary_list, "good_up_rates")

In [ ]:
bad_mean_list = show_mean_summary(summary_list=summary_list, target_key="bad_up_rates")

In [ ]:
sum(np.logical_and(good_mean_list < 0, -bad_mean_list < 0))

In [ ]:
len(good_mean_list)